In [ ]:
import telebot
from telebot import TeleBot
from telebot import types
from time import sleep
import os
import codecs
import logging
import config
import sqlite3
from datetime import datetime, timedelta
from dbworker import *
import numpy as np
import requests
import json
import time
from hashlib import sha256

logging.basicConfig(filename="botlog.log", level=logging.INFO, filemode='w')
token = '1618504886:AAFfCCV23hBgKqgDVScZkReEN7JntUevKdg'

logging.info('Started')

bot = TeleBot(token) 

mainMarkup = types.ReplyKeyboardMarkup(resize_keyboard=True)
bookCourt = types.KeyboardButton('Забронировать корт')
bookTraining = types.KeyboardButton('Записаться на тернировку')
bookGame = types.KeyboardButton('Записаться на игру')
mainMarkup.row(bookCourt)
mainMarkup.row(bookTraining)
mainMarkup.row(bookGame)

backMarkup = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
backBtn = types.KeyboardButton('Вернуться в главное меню')
backMarkup.add(backBtn)
    
removeMarkup = types.ReplyKeyboardRemove(selective=False)

request = {}

@bot.message_handler(func = lambda x: x.text == 'Вернуться в главное меню')
def main_menu(message):
    bot.send_message(message.chat.id, 'Главное меню', reply_markup=mainMarkup)
    return

@bot.message_handler(commands = ['start'])
def start_command(message):
    user = search_or_save_user(mdb, message.from_user)
    greetings = 'Привет, это бот Global Sport Arena. С помощью него можно записаться на тренировки, бронировать корты и узнавать о последних новостях'
    rm = mainMarkup
    if user['state'] == -2:
        greetings += '\n\nЯ вижу, Вы у нас в первый раз, давайте пройдем коротенькую процедуру регистрации. Укажите ваше имя для записи на события'
        rm = removeMarkup
        bot.register_next_step_handler(message, register)
    bot.send_message(message.chat.id, greetings, reply_markup=rm)
    return

def register(message):
    global request
    request[message.chat.id] = {}
    update_user_field(mdb, message.from_user.id, 'nick', message.text)
    set_state(mdb, message.from_user.id, 1)
    bot.send_message(message.chat.id, 'Вы успешно зарегистрированы. Можете начинать пользоваться ботом!', reply_markup=mainMarkup)
    return

@bot.message_handler(func = lambda x: ('Забронировать' in x.text or 'Записаться' in x.text) and get_current_state(mdb, x.from_user.id) > 0)
def booking_init(message):
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    global request
    request[message.chat.id] = {}
    if message.text == 'Забронировать корт':
        request[message.chat.id]['type'] = 0 
    elif message.chat.id == 'Записаться на тернировку':
        request[message.chat.id]['type'] = 1
    elif message.chat.id == 'Записаться на игру':
        request[message.chat.id]['type'] = 2
    else:
        bot.send_message(message.chat.id, 'Что то пошло не так, попробуйте позже', reply_markup=mainMarkup)
        return
    bot.send_message(message.chat.id, 'Пришлите желаемую дату в формате дд.мм.гг чч', reply_markup=backMarkup)
    bot.register_next_step_handler(message, get_date)
    return

def get_date(message):
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    global request
    try:
        date=datetime.strptime(message.text, "%d.%m.%y %H")
    except Exception as e:
        bot.send_message(message.chat.id, 'Неверный формат. Попробуйте в этом формате дд.мм.гг чч', reply_markup=backMarkup)
        bot.register_next_step_handler(message, get_date)
        return
    if date < datetime.now():
        bot.send_message(message.chat.id, 'Время уже прошло, попробуйте еще раз', reply_markup=backMarkup)
        bot.register_next_step_handler(message, get_date)
        return
    request[message.chat.id]['date'] = date
    bot.send_message(message.chat.id, 'Напишите продолжительность брони в часах, одним числом', reply_markup=backMarkup)
    bot.register_next_step_handler(message, get_booking_duration)
    return
    
def get_booking_duration(message):
    global request
    if message.text == 'Вернуться в главное меню':
        main_menu(message)
        return
    try:
        duration = int(message.text)
    except:
        bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
        bot.register_next_step_handler(message, get_booking_duration)
        return
    request[message.chat.id]['duration'] = duration
    start = request[message.chat.id]['date']
    end = start + timedelta(hours=duration-1)
    bookings = mdb.bookings.find({})
    courts = set([c['courtId'] for c in mdb.courts.find({})])
    for booking in bookings:
        if booking['start'] >= start and booking['start'] <= end or booking['end'] >= start and booking['end'] <= end:
            try:
                courts.remove(booking['courtId'])
            except KeyError:
                continue
    if len(courts) == 0:
        bot.send_message(message.chat.id, 'Текущий диапазон занят. Попробуйте бронировать на меньшее время. Введите новое число часов', reply_markup=backMarkup)
        bot.register_next_step_handler(message, get_booking_duration)
        return
    request[message.chat.id]['court'] = list(courts)[0]
    request[message.chat.id]['price'] = str(get_booking_cost(start, end))
    msg = 'Проверьте составленную бронь. Если все в порядке, подтвердите, или забронируйте заново\n\n'
    msg += '<strong>Корт</strong>: ' + str(request[message.chat.id]['court'])
    msg += '\n<strong>Дата и время начала брони</strong>: ' + str(start)
    msg += '\n<strong>Дата и время конца брони</strong>: ' + str(end + timedelta(hours=1))
    msg += '\n<strong>Имя бронирующего</strong>: ' + search_or_save_user(mdb, message.from_user)['nick']
    msg += '\n<strong>Стоимость</strong>: ' + request[message.chat.id]['price']
    approveMarkup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
    approveMarkup.row('Подтвердить')
    approveMarkup.row('Вернуться в главное меню')
    bot.send_message(message.chat.id, msg, reply_markup=approveMarkup, parse_mode='HTML')
    bot.register_next_step_handler(message, booking_approval)
    return

def booking_approval(message):
    global request
    booking = {}
    if message.text == 'Подтвердить':
        bookingId = np.random.randint(1, 10000)
        while check_booking(mdb, bookingId) is not None:
            bookingId = np.random.randint(1, 10000)
        booking['bookingId'] = bookingId
        booking['start'] = request[message.chat.id]['date']
        booking['duration'] = request[message.chat.id]['duration']
        booking['end'] = booking['start'] + timedelta(hours = request[message.chat.id]['duration'] - 1)
        booking['nick'] = search_or_save_user(mdb, message.from_user)['nick']
        booking['court'] = request[message.chat.id]['court']
        booking['price'] = request[message.chat.id]['price']
        print(booking)
        add_booking_to_court(mdb, booking['court'], bookingId)
        print(1)
        mdb.bookings.insert_one(booking)
        print(2)
        bot.send_message(message.chat.id, 'Бронь сохранена. Назовите ваше имя адиминистратору на арене для оплаты брони', reply_markup=mainMarkup)
    else:
        main_menu(message)
    return
    
    
    
def get_booking_cost(start, end):
    cost = 0
    def locc(dt, h):
        if dt.weekday() < 5:
            if h >=7 and h < 15:
                cost = 1500
            elif h >= 15 and h < 17:
                cost = 1700
            elif h >= 17 and h < 19:
                cost = 2500
            elif h >=19 and h < 23:
                cost = 3000
            elif h < 7:
                cost = 2500
            elif h == 23: 
                cost = 2500
        else:
            cost = 2700
        return cost
    if start.hour <= end.hour:
        for h in range(start.hour, end.hour + 1):
            cost += locc(start, h)
    else:
        for h in range(0, end.hour + 1):
            cost += locc(end, h)
        for h in range(start.hour, 24):
            cost += locc(start, h)
    return cost
    
bot.polling(none_stop=True)    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     courtMarkup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
#     courts = mdb.courts.find({})
#     for court in courts:
#         courtMarkup.row(types.KeyboardButton('Корт ' + str(court['courtId'])))
#     bot.send_message(message.chat.id, 'Выберите корт', reply_markup=courtMarkup)
#     bot.register_next_step_handler(message, court_choosing)
#     return

# def court_choosing(message):
#     global request
#     if message.text.split(' ')[0] != 'Корт':
#         bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
#         bot.register_next_step_handler(message, court_choosing)
#         return
#     request[message.chat.id]['court'] = message.text.split(' ')[1]
#     bookings = mdb.bookings.find({'court': request[message.chat.id]['court']})
#     busy = set()
#     for booking in bookings:
#         if booking['start'].hour <= booking['end'].hour:
#             if datetime(booking['start'].year, booking['start'].month, booking['start'].day) == request[message.chat.id]['date']:
#                 for h in range(booking['start'].hour, booking['end'].hour + 1):
#                     busy.add(h)
#         elif booking['start'].hour > booking['end'].hour:
#             if datetime(booking['start'].year, booking['start'].month, booking['start'].day) == request[message.chat.id]['date']:
#                 for h in range(booking['start'].hour, 24):
#                     busy.add(h)
#             elif datetime(booking['end'].year, booking['end'].month, booking['end'].day) == request[message.chat.id]['date']:
#                 for h in range(0, booking['end'].hour + 1):
#                     busy.add(h)
#     timeKeyboard = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=1)
#     free = []
#     for i in range(24):
#         if i not in busy:
#             tme = str(i)+':00'
#             if i < 10:
#                 tme = '0' + tme
#             timeKeyboard.row(types.KeyboardButton(tme))
#     bot.send_message(message.chat.id, 'Выберите время начала брони', reply_markup=timeKeyboard)
#     bot.register_next_step_handler(message, time_choosing)
#     return

# def time_choosing(message):
#     if message.text.split(':')[1] != '00':
#         bot.send_message(message.chat.id, 'Что-то пошло не так, попробуйте еще раз')
#         bot.register_next_step_handler(message, time_choosing)
#         return
#     global request
#     request[message.chat.id]['start_hour'] = int(message.text.split(':')[0])
#     bot.send_message(message.chat.id, 'Напишите продолжительность брони в часах', reply_markup=backMarkup)
#     bot.register_next_step_handler(message, get_booking_duration)
#     return
# def get_dayzone(message):
#     global request
#     if message.text == 'Утро':
#         request[message.chat.id]['dayzone'] = 0
#         check = [5,6,7,8,9,10]
#     elif message.text == 'День':
#         request[message.chat.id]['dayzone'] = 1
#         check = [11,12,13,14,15,16]
#     elif message.text == 'Вечер':
#         request[message.chat.id]['dayzone'] = 2
#         check = [17,18,19,20,21,22]
#     elif message.text == 'Ночь':
#         request[message.chat.id]['dayzone'] = 3
#         check = [23,0,1,2,3,4]
#     else:
#         bot.send_message(message.chat.id, 'Что то пошло не так, попробуйте позже', reply_markup=mainMarkup)
#         return
#     courts = mdb.courts.find({})
#     for i in range(len(courts)):
#         courts[i]['check'] = check
#         courts[i]['free'] = 6
#     courtids = [court['courtId'] for court in courts]
#     bookings = mdb.bookings.find({})
#     for booking in bookings:
#         if datetime(booking['start'].year, booking['start'].month, booking['start'].day) == request[message.chat.id]['date']:
#             for hour in check:
#                 if hour >= booking['start'].hour and hour < booking['start'].hour + booking['duration']:
#                     courts[booking['court']]['free'] -= 1
#         if datetime(booking['end'].year, booking['end'].month, booking['start'].day) == request[message.chat.id]['date']:
#             for hour in check:
#                 if hour >= booking['start'].hour and hour < booking['start'].hour + booking['duration']:
#                     courts[booking['court']]['free'] -= 1
                
#     bot.send_message(message.chat.id, 'Выберите корт')
#     bot.register_next_step_handler(message, court_selector)
    

{'bookingId': 1978, 'start': datetime.datetime(2021, 1, 27, 20, 0), 'duration': 8, 'end': datetime.datetime(2021, 1, 28, 3, 0), 'nick': 'Елисей', 'court': 0, 'price': '21500'}
1
2


In [14]:
import json

In [16]:
court = {'courtId' : 0, 'bookings': []}

In [20]:
with open('temp.json','w') as fout:
    json.dump(court, fout)

In [25]:
d = datetime.now()

In [27]:
d.weekday()

2